# Finetune Masked-AutoEncoder

- Finetune the deep network after pretraining the self-supervised learning framework.

-----

## Load Packages

In [ ]:
# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2
%cd ..

In [ ]:
# Load some packages
import os
import gc
from copy import deepcopy
import hydra
from omegaconf import OmegaConf
import wandb
import pprint
import numpy as np
import torch
from tqdm.auto import tqdm
from collections import OrderedDict
from cycler import cycler

import matplotlib
import matplotlib.pyplot as plt
import scienceplots

# custom package
from run_train import check_device_env
from run_train import set_seed
from run_train import compose_dataset
from run_train import generate_model
from train.ssl_train_script import ssl_train_script
from train.train_script import train_script
from models.utils import count_parameters

---

## Specify the dataset, model, and train setting

In [ ]:
pre_model_path = 'local/checkpoint/'
pre_model_name = 'eg6s5fay'  # '5xlos421'
device = 'cuda:1'

############
# Artifact #
############
art_config = dict()
art_config['project'] = 'caueeg-mae-artifact'
art_config['use_wandb'] = False
art_config['pre_model'] = pre_model_name
art_config['device'] = device

# art_config["art_filter_list"] = [9, 9, 9, 9, 9]
art_config["art_dropout"] = 0.1
art_config["art_use_age"] = "no"  # "conv", "embedding", "no"
art_config["art_out_activation"] = "softplus"  # "none", "relu", "softplus"  ######################################
art_config["art_loss_type"] = "mse"

art_config['total_samples'] = 3.0e+6
art_config['search_lr'] = False
art_config['base_lr'] = 1e-4
art_config['lr_scheduler_type'] = 'cosine_decay_with_warmup_half'

art_config["warmup_min"] = 150
art_config["num_history"] = 50
art_config['save_model'] = False

##################
# Classification #
##################
finetune_config = dict()
finetune_config['project'] = 'caueeg-abnormal-mae-artifact-finetune'
finetune_config['use_wandb'] = True
finetune_config['pre_model'] = pre_model_name
finetune_config['device'] = device
finetune_config["task"] = "abnormal"
finetune_config["out_dims"] = 3

# finetune_config["mask_ratio"] = 0.25
finetune_config["descending"] = False  #######################################################
finetune_config["global_pool"] = True
finetune_config["fc_stages"] = 2
finetune_config["dropout"] = 0.1
finetune_config["use_age"] = "conv"  ##################################################
# finetune_config["mixup"] = 0.3 ###
# finetune_config["crop_length"] = 8192*4  #############################################################
# finetune_config["criterion"] = "multi-bce"  # "cross-entropy", "multi-bce"

finetune_config["tuning_type"] = "finetune"  # "finetune", "fc_stage"
finetune_config["layer_wise_lr"] = True

finetune_config['total_samples'] = 3.0e+6  ############################################################
# finetune_config['base_lr'] = 1e-3
finetune_config['search_lr'] = True
finetune_config['lr_search_steps'] = 100
finetune_config['lr_scheduler_type'] = 'cosine_decay_with_warmup_half'
finetune_config["warmup_min"] = 200
finetune_config["num_history"] = 50
finetune_config['save_model'] = True

In [ ]:
finetune_cycler = cycler(mask_ratio=[0.3])
finetune_cycler *= cycler(crop_length=[2560 * 2]) + cycler(minibatch=[128])
finetune_cycler *= cycler(mixup=[0.1, 0.3])
finetune_cycler *= cycler(total_samples=[1.0e+7, 2.0e+7])

# finetune_cycler *= cycler(seed=[0, 1, 2])
finetune_cycler *= cycler(descending=[False])
for cyc in finetune_cycler:
    print(cyc)

In [ ]:
print('PyTorch version:', torch.__version__)
device = torch.device(device if torch.cuda.is_available() else 'cpu')

if torch.cuda.is_available(): print('cuda is available.')
else: print('cuda is unavailable.')

---
## Uncertainty Pretraining

In [ ]:
# load pretrained configurations
path = os.path.join(pre_model_path, pre_model_name.split(',')[-1], 'checkpoint.pt')
try:
    ckpt = torch.load(path, map_location="cpu")
    config = ckpt['config']
except Exception as e:
    print(e)
    print(f'- checkpoint cannot be opened: {path}')
config["cwd"] = ""

pprint.pprint(config)

In [ ]:
# update configuration
for k, v in art_config.items():
    config[k] = v

In [ ]:
# load pretrained model
if not config.get("ddp", False):
    pre_model_state = ckpt["ssl_model_state"]
else:
    pre_model_state_ddp = deepcopy(ckpt["ssl_model_state"])
    pre_model_state = OrderedDict()
    for k, v in pre_model_state_ddp.items():
        name = k[7:]  # remove 'module.' of DataParallel/DistributedDataParallel
        pre_model_state[name] = v

In [ ]:
config["ddp"] = False

# check the workstation environment and update some configurations
check_device_env(config)

# compose dataset
train_loader, val_loader, test_loader, multicrop_test_loader = compose_dataset(config)

pprint.pprint(config)

In [ ]:
# generate the model
config["_target_"] = config["_target_"].replace('pre', 'pre_art').replace('.mae_1d.', '.mae_1d_artifact.')
model = generate_model(config).to(device)

# load the model
model_state = model.state_dict()
for k, v in model_state.items():
    if not k.startswith('art') and not k.endswith("pos_embed"):
        model_state[k] = pre_model_state[k]

pre_model_state = deepcopy(model.state_dict())
model.load_state_dict(model_state)

In [ ]:
model.requires_grad_(False)
model = model.eval()

model.art_net.requires_grad_(True)
for k, v in model._parameters.items():
    if k.startswith("art"):
        v.requires_grad_(True)
model.art_net = model.art_net.train()

config["num_params"] = count_parameters(model)

# print(model)

In [ ]:
for name, param in model.named_parameters():
    print(f"{name:100}\t|\t{param.requires_grad}")

In [ ]:
# collect some garbage
gc.collect()
torch.cuda.empty_cache()
torch.cuda.synchronize()

# fix the seed for reproducibility (a negative seed value means not fixing)
set_seed(config, rank=None)

# train
ssl_train_script(
    config,
    model,
    train_loader,
    config["preprocess_train"],
)

for k, v in model_state.items():
    pre_model_state[k] = model_state[k].to("cpu")

---
## Visualize

In [ ]:
signal_header = [channel.split('-')[0] for i, channel in enumerate(config["signal_header"])]
fps = config.get('resample', 200)

In [ ]:
@plt.style.context(['ieee', 'science', 'default'])
def draw_eeg_graph(x, pred, mask, art_out, rec_loss, num=None):
    plt.rcParams.update({'font.family': 'Ubuntu'})
    N, C, L = x.shape
    _, l = mask.shape
    patch_size = L // l

    art_out = art_out.reshape(N, -1)
    rec_loss = rec_loss.reshape(N, -1)

    for n in range(N):
        if num is not None and num <= n:
            break
        
        fig = plt.figure(num=1, clear=True, figsize=(25.0, 17.0))
        fig.subplots_adjust(hspace=0)
        
        for c in range(C):
            ax = fig.add_subplot(C, 1, c + 1)
            ax.plot(x[n, c].cpu().numpy(), lw=1, c='tab:red', label='origin')
            ax.plot(pred[n, c].cpu().numpy(), lw=1, c='tab:blue', label='pred')

            art_mid = art_out[n].median()
            rec_mid = art_out[n].median()
            for r in range(l):
                if r > 0:
                    ax.axvline(r*patch_size, color='tab:purple', alpha=0.4)
                if mask[n, r]:
                    ax.axvspan(r*patch_size, (r + 1)*patch_size, facecolor='tab:purple', alpha=0.2)

                if c == 0:
                    if art_mid <= art_out[n, r]:
                        ax.annotate(f"{art_out[n, r]:3.2f}", 
                                    xy=((r + 0.5)*patch_size, 1), ha='center', va='bottom', color='tab:purple')
                    else:
                        ax.annotate(f"{art_out[n, r]:3.2f}", 
                                    xy=((r + 0.5)*patch_size, 1), ha='center', va='bottom')
                    if rec_mid <= rec_loss[n, r]:
                        ax.annotate(f"{rec_loss[n, r]:3.2f}", 
                                    xy=((r + 0.5)*patch_size, -1), ha='center', va='bottom', color='tab:purple')
                    else:
                        ax.annotate(f"{rec_loss[n, r]:3.2f}", 
                                    xy=((r + 0.5)*patch_size, -1), ha='center', va='bottom')

            ax.set_xlim(0, L)
            ax.set_ylabel(signal_header[c])
            ax.set_xticks(np.arange(round(config["seq_length"] / fps) + 1) * fps)
            ax.set_xticklabels([])
            # ax.tick_params(axis='x', width=0.1, length=0.1)
            ax.set_yticks([0])
            ax.set_yticklabels([])
        
        ax.set_xticks(np.arange(round(config["seq_length"] / fps) + 1) * fps)
        ax.set_xticklabels(np.arange(round(config["seq_length"] / fps) + 1))
        
        ax.set_xlabel('Time (s)')
        # fig.savefig(os.path.join(output_folder, 'signal_example.pdf'), transparent=True)
        plt.show()
        fig.clear()
        plt.close(fig)

In [ ]:
with torch.no_grad():
    for target_dataset in tqdm(["val"], desc="Dataset", leave=False):
        if target_dataset == 'train':
            loader = train_loader
        elif target_dataset == 'val':
            loader = val_loader
        elif target_dataset == 'test':
            loader = test_loader
        else:
            raise ValueError('')
                
        for sample_batched in tqdm(loader, total=len(loader), desc='Batch', leave=False):
            print(target_dataset)
            config["preprocess_test"](sample_batched)
            x = sample_batched["signal"]
            age = sample_batched["age"]

            pred, mask = model.mask_and_reconstruct(x, age, config["mask_ratio"])
            rec_loss = model.compute_reconstruction_loss2(x, pred)
            art_out = model.forward_artifact(x, age)
            
            pred_eeg = model.unpatchify(pred)
            draw_eeg_graph(x, pred_eeg, mask, art_out, rec_loss, 2)

            break
        break

In [ ]:
del model, ckpt

gc.collect()
torch.cuda.empty_cache()
torch.cuda.synchronize()

---
## Finetuning

In [ ]:
for cyc in finetune_cycler:
    # update configuration
    for k, v in finetune_config.items():
        config[k] = v
    for k, v in cyc.items():
        config[k] = v
    config["ddp"] = False

    # check the workstation environment and update some configurations
    check_device_env(config)
    
    # compose dataset
    train_loader, val_loader, test_loader, multicrop_test_loader = compose_dataset(config)
    pprint.pprint(config)

    # generate the model
    config["_target_"] = config["_target_"].replace('.ssl', '').replace('_pre', '')
    model = generate_model(config).to(device)
    
    # load the model
    model_state = model.state_dict()
    for k, v in model_state.items():
        if not k.startswith("fc") and not k.endswith("pos_embed"):
            model_state[k] = pre_model_state[k]
    
    model.load_state_dict(model_state)
    model.finetune_mode(config["tuning_type"])
    config["num_params"] = count_parameters(model)
    
    for name, param in model.named_parameters():
        print(f"{name:100}\t|\t{param.requires_grad}")

    # collect some garbage
    gc.collect()
    torch.cuda.empty_cache()
    torch.cuda.synchronize()
    
    # fix the seed for reproducibility (a negative seed value means not fixing)
    set_seed(config, rank=None)
    
    # train
    train_script(
        config,
        model,
        train_loader,
        val_loader,
        test_loader,
        multicrop_test_loader,
        config["preprocess_train"],
        config["preprocess_test"],
    )

print("- END -")